In [3]:
import rasterio as rio
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import mean_squared_error
import seaborn as sns
from functools import reduce

In [4]:
def scaled(x):
    min_val, max_val = np.percentile(x, (2, 98))

    # Scale the pixel values to the range of 0-255
    return np.interp(x, (min_val, max_val), (0, 255)).astype(np.uint8)

def sigmoid(x):
    return 255 / (1 + np.exp(-4 * (scaled(x) / 255 - 0.5)))

def get_detect_cloud_bands(img):
    bands = [sigmoid(img.read(b)) for b in img.indexes]
    bands = np.stack(bands)
    return bands.transpose(1,2,0).astype(np.uint)
    

In [5]:
path = Path('../../imgs')
images = {p.name: p  for p in path.iterdir()}
path_imgs = [im for im in images['s2'].iterdir()]
rio_cloudless = [get_detect_cloud_bands(rio.open(path)) for path in path_imgs]
path_imgs = [im for im in images['s2_cloudy'].iterdir()]
rio_cloudies = [get_detect_cloud_bands(rio.open(path)) for path in path_imgs]

In [6]:
rio_cloudless[0]

array([[[ 86, 128,  98, ..., 106,  51,  75],
        [ 91, 110,  97, ...,  67,  49,  77],
        [ 91,  99, 101, ...,  67,  50,  78],
        ...,
        [109, 127, 148, ..., 106, 102, 117],
        [109, 148, 143, ..., 106, 104, 115],
        [109, 131, 153, ..., 106, 110, 109]],

       [[ 84,  79,  88, ..., 106,  55,  84],
        [ 91,  99,  84, ...,  67,  50,  79],
        [ 91, 110,  89, ...,  67,  50,  79],
        ...,
        [109, 114, 146, ..., 106, 109, 117],
        [109, 119, 155, ..., 106, 110, 116],
        [109, 133, 157, ..., 106, 114, 110]],

       [[ 84,  79,  91, ..., 106,  62,  97],
        [ 91,  92,  86, ...,  67,  53,  85],
        [ 91, 120,  90, ...,  67,  52,  82],
        ...,
        [115, 120, 152, ..., 106, 109, 118],
        [118,  97, 149, ..., 106, 110, 117],
        [118, 129, 158, ..., 106, 114, 112]],

       ...,

       [[174, 161, 158, ..., 127, 163, 172],
        [153, 166, 178, ..., 106, 165, 171],
        [150, 165, 184, ..., 106, 168, 172

In [19]:
def props(img):
    res = {}
    for b in range(img.shape[-1]):
        band_str = 'band_{}'.format(b+1)
        band_dict = {}
        band_dict[f'mean'] = img[:, :, b].mean()
        band_dict[f'median'] = np.percentile(img[:, :, b], 50)
        band_dict[f'p25'] = np.percentile(img[:, :, b], 25)
        band_dict[f'p75'] = np.percentile(img[:, :, b], 75)
        band_dict[f'std'] = img[:, :, b].std()
        res[band_str] = band_dict
    return res
props(rio_cloudless[0])

{'band_1': {'mean': 107.60281372070312,
  'median': 100.0,
  'p25': 79.0,
  'p75': 135.0,
  'std': 44.29389857882789},
 'band_2': {'mean': 119.41436767578125,
  'median': 117.0,
  'p25': 87.0,
  'p75': 151.0,
  'std': 47.288141497469915},
 'band_3': {'mean': 117.61798095703125,
  'median': 109.0,
  'p25': 80.0,
  'p75': 156.0,
  'std': 50.50850519319692},
 'band_4': {'mean': 118.9580078125,
  'median': 120.0,
  'p25': 90.0,
  'p75': 147.0,
  'std': 45.595466889966175},
 'band_5': {'mean': 98.98638916015625,
  'median': 83.0,
  'p25': 60.0,
  'p75': 130.0,
  'std': 49.979488263867665},
 'band_6': {'mean': 84.59135437011719,
  'median': 64.0,
  'p25': 42.0,
  'p75': 119.0,
  'std': 52.38822403147514},
 'band_7': {'mean': 83.18063354492188,
  'median': 64.0,
  'p25': 40.0,
  'p75': 115.0,
  'std': 51.963664881896044},
 'band_8': {'mean': 83.18502807617188,
  'median': 62.0,
  'p25': 40.0,
  'p75': 113.0,
  'std': 52.70838137571431},
 'band_9': {'mean': 83.99592590332031,
  'median': 67.0,